# OAuth 2.0 & OpenID Connect Deep Dive

A comprehensive guide to modern authentication and authorization protocols.

## Table of Contents
1. [OAuth 2.0 Overview](#oauth2-overview)
2. [Grant Types](#grant-types)
3. [OpenID Connect](#openid-connect)
4. [Token Handling](#token-handling)
5. [Security Best Practices](#security-best-practices)
6. [Python Implementation Examples](#python-examples)

---

## 1. OAuth 2.0 Overview <a id='oauth2-overview'></a>

**OAuth 2.0** is an authorization framework that enables third-party applications to obtain limited access to a user's resources without exposing their credentials.

### Key Concepts

| Role | Description |
|------|-------------|
| **Resource Owner** | The user who owns the data and grants access |
| **Client** | The application requesting access to resources |
| **Authorization Server** | Issues tokens after authenticating the resource owner |
| **Resource Server** | Hosts the protected resources (APIs) |

### OAuth 2.0 vs OAuth 1.0

| Feature | OAuth 1.0 | OAuth 2.0 |
|---------|-----------|----------|
| Signature | Required (HMAC-SHA1) | Not required (uses TLS) |
| Token Types | Single token | Access + Refresh tokens |
| Complexity | Higher | Lower |
| Mobile Support | Limited | Excellent |
| Grant Types | Single flow | Multiple flows |

---

## 2. Grant Types <a id='grant-types'></a>

OAuth 2.0 defines several grant types for different use cases.

### 2.1 Authorization Code Grant

The most secure and commonly used flow for server-side applications.

```
┌─────────┐                              ┌───────────────┐
│  User   │                              │Authorization  │
│(Browser)│                              │    Server     │
└────┬────┘                              └───────┬───────┘
     │                                           │
     │ 1. Click "Login"                          │
     ├──────────────────────────────────────────►│
     │                                           │
     │ 2. Redirect to Auth Server                │
     │◄──────────────────────────────────────────┤
     │                                           │
     │ 3. User authenticates                     │
     ├──────────────────────────────────────────►│
     │                                           │
     │ 4. Authorization Code                     │
     │◄──────────────────────────────────────────┤
     │                                           │
┌────┴────┐                                      │
│  Client │                                      │
│ (Server)│                                      │
└────┬────┘                                      │
     │ 5. Exchange Code for Tokens               │
     ├──────────────────────────────────────────►│
     │                                           │
     │ 6. Access Token + Refresh Token           │
     │◄──────────────────────────────────────────┤
```

**Authorization Request Parameters:**
- `response_type=code`
- `client_id` - Application identifier
- `redirect_uri` - Where to send the user after authorization
- `scope` - Requested permissions
- `state` - CSRF protection token

In [ ]:
# Authorization Code Grant - Step 1: Build Authorization URL
import urllib.parse
import secrets

def build_authorization_url(
    auth_endpoint: str,
    client_id: str,
    redirect_uri: str,
    scope: str,
    state: str = None
) -> str:
    """
    Build the authorization URL for the Authorization Code flow.
    
    Args:
        auth_endpoint: Authorization server's authorization endpoint
        client_id: Your application's client ID
        redirect_uri: Where to redirect after authorization
        scope: Space-separated list of requested scopes
        state: Optional CSRF protection token (generated if not provided)
    
    Returns:
        Complete authorization URL
    """
    if state is None:
        state = secrets.token_urlsafe(32)
    
    params = {
        'response_type': 'code',
        'client_id': client_id,
        'redirect_uri': redirect_uri,
        'scope': scope,
        'state': state
    }
    
    query_string = urllib.parse.urlencode(params)
    return f"{auth_endpoint}?{query_string}", state

# Example usage
auth_url, state = build_authorization_url(
    auth_endpoint='https://auth.example.com/authorize',
    client_id='my-app-client-id',
    redirect_uri='https://myapp.com/callback',
    scope='openid profile email'
)

print(f"Authorization URL:\n{auth_url}")
print(f"\nState (store in session): {state}")

In [ ]:
# Authorization Code Grant - Step 2: Exchange Code for Tokens
import requests
from typing import Dict, Any

def exchange_code_for_tokens(
    token_endpoint: str,
    client_id: str,
    client_secret: str,
    code: str,
    redirect_uri: str
) -> Dict[str, Any]:
    """
    Exchange authorization code for access and refresh tokens.
    
    Args:
        token_endpoint: Authorization server's token endpoint
        client_id: Your application's client ID
        client_secret: Your application's client secret
        code: The authorization code received from callback
        redirect_uri: Must match the original redirect_uri
    
    Returns:
        Token response containing access_token, refresh_token, etc.
    """
    payload = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'code': code,
        'redirect_uri': redirect_uri
    }
    
    response = requests.post(
        token_endpoint,
        data=payload,
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    
    response.raise_for_status()
    return response.json()

# Example response structure
example_token_response = {
    'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9...',
    'token_type': 'Bearer',
    'expires_in': 3600,
    'refresh_token': 'dGhpcyBpcyBhIHJlZnJlc2ggdG9rZW4...',
    'scope': 'openid profile email',
    'id_token': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9...'  # If OIDC
}

print("Example Token Response:")
for key, value in example_token_response.items():
    print(f"  {key}: {value[:50]}..." if len(str(value)) > 50 else f"  {key}: {value}")

### 2.2 Authorization Code with PKCE (Proof Key for Code Exchange)

**PKCE** (pronounced "pixy") is an extension to the Authorization Code flow that adds additional security, especially for public clients (mobile apps, SPAs) that cannot securely store a client secret.

```
┌─────────────────────────────────────────────────────────────┐
│                    PKCE Flow                                 │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  1. Generate code_verifier (random string, 43-128 chars)    │
│     code_verifier = "dBjftJeZ4CVP-mB92K27uhbUJU1p1r_wW1gFWF.."│
│                                                              │
│  2. Create code_challenge = BASE64URL(SHA256(code_verifier))│
│     code_challenge = "E9Melhoa2OwvFrEMTJguCHaoeK1t8URWbuGJ.." │
│                                                              │
│  3. Send code_challenge in authorization request             │
│                                                              │
│  4. Send code_verifier in token exchange request             │
│                                                              │
│  5. Server verifies: SHA256(code_verifier) == code_challenge │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

**Why PKCE?**
- Prevents authorization code interception attacks
- No need to store client secrets on public clients
- Recommended for ALL OAuth 2.0 clients (RFC 7636)

In [ ]:
# PKCE Implementation
import hashlib
import base64
import secrets
import urllib.parse

class PKCEHelper:
    """
    Helper class for PKCE (Proof Key for Code Exchange) operations.
    """
    
    @staticmethod
    def generate_code_verifier(length: int = 64) -> str:
        """
        Generate a cryptographically random code verifier.
        
        Args:
            length: Length of the verifier (43-128 characters)
        
        Returns:
            URL-safe code verifier string
        """
        if not 43 <= length <= 128:
            raise ValueError("code_verifier length must be between 43 and 128")
        
        # Generate random bytes and encode as URL-safe base64
        random_bytes = secrets.token_bytes(length)
        code_verifier = base64.urlsafe_b64encode(random_bytes).decode('utf-8')
        # Remove padding and limit to specified length
        return code_verifier.rstrip('=')[:length]
    
    @staticmethod
    def generate_code_challenge(code_verifier: str, method: str = 'S256') -> str:
        """
        Generate code challenge from code verifier.
        
        Args:
            code_verifier: The code verifier string
            method: Challenge method ('S256' or 'plain')
        
        Returns:
            Code challenge string
        """
        if method == 'S256':
            # SHA256 hash of verifier, then base64url encode
            digest = hashlib.sha256(code_verifier.encode('utf-8')).digest()
            return base64.urlsafe_b64encode(digest).decode('utf-8').rstrip('=')
        elif method == 'plain':
            # Plain method (not recommended)
            return code_verifier
        else:
            raise ValueError(f"Unsupported method: {method}")
    
    @staticmethod
    def build_authorization_url_with_pkce(
        auth_endpoint: str,
        client_id: str,
        redirect_uri: str,
        scope: str,
        code_challenge: str,
        code_challenge_method: str = 'S256',
        state: str = None
    ) -> tuple:
        """
        Build authorization URL with PKCE parameters.
        
        Returns:
            Tuple of (authorization_url, state)
        """
        if state is None:
            state = secrets.token_urlsafe(32)
        
        params = {
            'response_type': 'code',
            'client_id': client_id,
            'redirect_uri': redirect_uri,
            'scope': scope,
            'state': state,
            'code_challenge': code_challenge,
            'code_challenge_method': code_challenge_method
        }
        
        query_string = urllib.parse.urlencode(params)
        return f"{auth_endpoint}?{query_string}", state


# Example: Complete PKCE flow
pkce = PKCEHelper()

# Step 1: Generate code verifier (store securely)
code_verifier = pkce.generate_code_verifier()
print(f"Code Verifier (keep secret): {code_verifier}")

# Step 2: Generate code challenge
code_challenge = pkce.generate_code_challenge(code_verifier)
print(f"Code Challenge (send in auth request): {code_challenge}")

# Step 3: Build authorization URL
auth_url, state = pkce.build_authorization_url_with_pkce(
    auth_endpoint='https://auth.example.com/authorize',
    client_id='my-spa-client-id',
    redirect_uri='https://myapp.com/callback',
    scope='openid profile email',
    code_challenge=code_challenge
)
print(f"\nAuthorization URL:\n{auth_url}")

In [ ]:
# PKCE Token Exchange (no client_secret required)
def exchange_code_for_tokens_pkce(
    token_endpoint: str,
    client_id: str,
    code: str,
    redirect_uri: str,
    code_verifier: str
) -> dict:
    """
    Exchange authorization code for tokens using PKCE.
    No client_secret is required for public clients.
    
    Args:
        token_endpoint: Authorization server's token endpoint
        client_id: Your application's client ID
        code: The authorization code from callback
        redirect_uri: Must match original redirect_uri
        code_verifier: The original code verifier
    
    Returns:
        Token response dictionary
    """
    payload = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'code': code,
        'redirect_uri': redirect_uri,
        'code_verifier': code_verifier  # This proves we initiated the request
    }
    
    response = requests.post(
        token_endpoint,
        data=payload,
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    
    response.raise_for_status()
    return response.json()

print("PKCE Token Exchange - No client_secret needed!")
print("Server validates: SHA256(code_verifier) == stored_code_challenge")

### 2.3 Client Credentials Grant

Used for **machine-to-machine (M2M)** communication where no user is involved.

```
┌──────────┐                           ┌───────────────┐
│  Client  │                           │ Authorization │
│ (Server) │                           │    Server     │
└────┬─────┘                           └───────┬───────┘
     │                                         │
     │  POST /token                            │
     │  grant_type=client_credentials          │
     │  client_id=xxx                          │
     │  client_secret=xxx                      │
     │  scope=xxx                              │
     ├────────────────────────────────────────►│
     │                                         │
     │  { access_token: "...", expires_in: ...}│
     │◄────────────────────────────────────────┤
     │                                         │
```

**Use Cases:**
- Microservices communication
- Background jobs/cron tasks
- CLI tools
- Server-to-server API calls

In [ ]:
# Client Credentials Grant Implementation
import requests
from datetime import datetime, timedelta
from typing import Optional
import threading

class ClientCredentialsAuth:
    """
    OAuth 2.0 Client Credentials Grant implementation with token caching.
    """
    
    def __init__(
        self,
        token_endpoint: str,
        client_id: str,
        client_secret: str,
        scope: str = None,
        token_buffer_seconds: int = 60
    ):
        """
        Initialize the client credentials authenticator.
        
        Args:
            token_endpoint: OAuth token endpoint URL
            client_id: Client application ID
            client_secret: Client secret
            scope: Optional scope string
            token_buffer_seconds: Refresh token this many seconds before expiry
        """
        self.token_endpoint = token_endpoint
        self.client_id = client_id
        self.client_secret = client_secret
        self.scope = scope
        self.token_buffer = timedelta(seconds=token_buffer_seconds)
        
        self._access_token: Optional[str] = None
        self._token_expiry: Optional[datetime] = None
        self._lock = threading.Lock()
    
    def _fetch_token(self) -> dict:
        """
        Fetch a new access token from the authorization server.
        """
        payload = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        
        if self.scope:
            payload['scope'] = self.scope
        
        response = requests.post(
            self.token_endpoint,
            data=payload,
            headers={'Content-Type': 'application/x-www-form-urlencoded'}
        )
        
        response.raise_for_status()
        return response.json()
    
    def get_access_token(self) -> str:
        """
        Get a valid access token, refreshing if necessary.
        Thread-safe implementation.
        
        Returns:
            Valid access token string
        """
        with self._lock:
            # Check if we need a new token
            if self._access_token is None or self._is_token_expired():
                token_response = self._fetch_token()
                
                self._access_token = token_response['access_token']
                expires_in = token_response.get('expires_in', 3600)
                self._token_expiry = datetime.now() + timedelta(seconds=expires_in)
            
            return self._access_token
    
    def _is_token_expired(self) -> bool:
        """
        Check if the token is expired or will expire soon.
        """
        if self._token_expiry is None:
            return True
        return datetime.now() + self.token_buffer >= self._token_expiry
    
    def get_auth_header(self) -> dict:
        """
        Get the Authorization header for API requests.
        
        Returns:
            Dictionary with Authorization header
        """
        return {'Authorization': f'Bearer {self.get_access_token()}'}


# Example usage
print("Client Credentials Grant - Machine-to-Machine Authentication")
print("="*60)

# Initialize (would use real credentials in production)
auth = ClientCredentialsAuth(
    token_endpoint='https://auth.example.com/oauth/token',
    client_id='my-service-client-id',
    client_secret='my-service-client-secret',
    scope='api:read api:write'
)

print("\nUsage example:")
print("""
# Make authenticated API call
response = requests.get(
    'https://api.example.com/data',
    headers=auth.get_auth_header()
)
""")

### 2.4 Grant Types Comparison

| Grant Type | Use Case | Client Type | User Involved |
|------------|----------|-------------|---------------|
| **Authorization Code** | Web apps with backend | Confidential | Yes |
| **Authorization Code + PKCE** | SPAs, Mobile apps | Public | Yes |
| **Client Credentials** | M2M, Services | Confidential | No |
| **Refresh Token** | Token renewal | Both | No (after initial auth) |

**Deprecated Grants (OAuth 2.1):**
- ❌ **Implicit Grant** - Replaced by Authorization Code + PKCE
- ❌ **Resource Owner Password Credentials** - Security concerns

---

## 3. OpenID Connect (OIDC) <a id='openid-connect'></a>

**OpenID Connect** is an identity layer built on top of OAuth 2.0. While OAuth 2.0 handles authorization, OIDC handles **authentication**.

### Key Differences

| Aspect | OAuth 2.0 | OpenID Connect |
|--------|-----------|----------------|
| Purpose | Authorization | Authentication + Authorization |
| Token | Access Token | Access Token + **ID Token** |
| User Info | Not standardized | Standardized claims |
| Discovery | Not defined | Well-known configuration |

### OIDC Scopes

| Scope | Claims Included |
|-------|----------------|
| `openid` | Required for OIDC, returns `sub` claim |
| `profile` | `name`, `family_name`, `given_name`, `picture`, etc. |
| `email` | `email`, `email_verified` |
| `phone` | `phone_number`, `phone_number_verified` |
| `address` | `address` (structured) |

In [ ]:
# OIDC Discovery - Fetching Provider Configuration
import requests
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class OIDCConfiguration:
    """
    OpenID Connect Provider Configuration.
    """
    issuer: str
    authorization_endpoint: str
    token_endpoint: str
    userinfo_endpoint: str
    jwks_uri: str
    scopes_supported: List[str]
    response_types_supported: List[str]
    grant_types_supported: List[str]
    id_token_signing_alg_values_supported: List[str]
    end_session_endpoint: Optional[str] = None
    revocation_endpoint: Optional[str] = None
    introspection_endpoint: Optional[str] = None


def discover_oidc_configuration(issuer_url: str) -> OIDCConfiguration:
    """
    Discover OIDC provider configuration from well-known endpoint.
    
    Args:
        issuer_url: The OIDC issuer URL (e.g., 'https://accounts.google.com')
    
    Returns:
        OIDCConfiguration object with provider endpoints
    """
    well_known_url = f"{issuer_url.rstrip('/')}/.well-known/openid-configuration"
    
    response = requests.get(well_known_url)
    response.raise_for_status()
    config = response.json()
    
    return OIDCConfiguration(
        issuer=config['issuer'],
        authorization_endpoint=config['authorization_endpoint'],
        token_endpoint=config['token_endpoint'],
        userinfo_endpoint=config['userinfo_endpoint'],
        jwks_uri=config['jwks_uri'],
        scopes_supported=config.get('scopes_supported', []),
        response_types_supported=config.get('response_types_supported', []),
        grant_types_supported=config.get('grant_types_supported', []),
        id_token_signing_alg_values_supported=config.get('id_token_signing_alg_values_supported', []),
        end_session_endpoint=config.get('end_session_endpoint'),
        revocation_endpoint=config.get('revocation_endpoint'),
        introspection_endpoint=config.get('introspection_endpoint')
    )


# Example: Google's OIDC configuration
print("OIDC Discovery Example")
print("="*50)
print("\nWell-known URL format:")
print("  {issuer}/.well-known/openid-configuration")
print("\nExample issuers:")
print("  - Google: https://accounts.google.com")
print("  - Microsoft: https://login.microsoftonline.com/{tenant}/v2.0")
print("  - Auth0: https://{domain}")
print("  - Okta: https://{org}.okta.com")

In [ ]:
# ID Token Structure and Validation
import json
import base64
from datetime import datetime

# Example ID Token (JWT format)
EXAMPLE_ID_TOKEN = """
eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImFiYzEyMyJ9.
eyJpc3MiOiJodHRwczovL2F1dGguZXhhbXBsZS5jb20iLCJzdWIiOiJ1c2VyXzEyMzQ1Niis
ImF1ZCI6Im15LWNsaWVudC1pZCIsImV4cCI6MTcwNDEwMDAwMCwiaWF0IjoxNzA0MDk2NDAwL
CJub25jZSI6InJhbmRvbS1ub25jZS12YWx1ZSIsIm5hbWUiOiJKb2huIERvZSIsImVtYWlsIj
oiam9obkBleGFtcGxlLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlfQ.
SIGNATURE_HERE
"""

def decode_jwt_payload(token: str) -> dict:
    """
    Decode JWT payload without verification (for inspection only).
    
    WARNING: Always verify tokens before trusting claims!
    
    Args:
        token: JWT token string
    
    Returns:
        Decoded payload as dictionary
    """
    parts = token.split('.')
    if len(parts) != 3:
        raise ValueError("Invalid JWT format")
    
    # Decode payload (second part)
    payload_b64 = parts[1]
    # Add padding if needed
    padding = 4 - len(payload_b64) % 4
    if padding != 4:
        payload_b64 += '=' * padding
    
    payload_bytes = base64.urlsafe_b64decode(payload_b64)
    return json.loads(payload_bytes)


# Standard ID Token Claims
print("ID Token Standard Claims")
print("="*50)

id_token_claims = {
    # Required claims
    'iss': 'https://auth.example.com',  # Issuer
    'sub': 'user_123456',                # Subject (unique user ID)
    'aud': 'my-client-id',               # Audience (your client ID)
    'exp': 1704100000,                   # Expiration time
    'iat': 1704096400,                   # Issued at time
    
    # Optional but common claims
    'auth_time': 1704096300,             # When user authenticated
    'nonce': 'random-nonce-value',       # Replay protection
    'acr': '0',                          # Authentication context class
    'amr': ['pwd', 'mfa'],               # Authentication methods used
    'azp': 'my-client-id',               # Authorized party
    
    # Profile claims (from 'profile' scope)
    'name': 'John Doe',
    'given_name': 'John',
    'family_name': 'Doe',
    'picture': 'https://example.com/photo.jpg',
    
    # Email claims (from 'email' scope)
    'email': 'john@example.com',
    'email_verified': True
}

print("\nRequired Claims:")
for claim in ['iss', 'sub', 'aud', 'exp', 'iat']:
    print(f"  {claim}: {id_token_claims[claim]}")

print("\nProfile Claims:")
for claim in ['name', 'email', 'email_verified']:
    print(f"  {claim}: {id_token_claims[claim]}")

In [ ]:
# UserInfo Endpoint
import requests
from typing import Dict, Any

def get_userinfo(userinfo_endpoint: str, access_token: str) -> Dict[str, Any]:
    """
    Fetch user information from the OIDC UserInfo endpoint.
    
    Args:
        userinfo_endpoint: The UserInfo endpoint URL
        access_token: Valid access token with appropriate scopes
    
    Returns:
        User information claims
    """
    response = requests.get(
        userinfo_endpoint,
        headers={'Authorization': f'Bearer {access_token}'}
    )
    
    response.raise_for_status()
    return response.json()


# Example UserInfo response
example_userinfo = {
    'sub': 'user_123456',
    'name': 'John Doe',
    'given_name': 'John',
    'family_name': 'Doe',
    'preferred_username': 'johndoe',
    'email': 'john@example.com',
    'email_verified': True,
    'picture': 'https://example.com/photos/johndoe.jpg',
    'locale': 'en-US',
    'updated_at': 1704096400
}

print("UserInfo Endpoint Response")
print("="*50)
print("\nEndpoint: GET /userinfo")
print("Header: Authorization: Bearer {access_token}")
print("\nExample Response:")
for key, value in example_userinfo.items():
    print(f"  {key}: {value}")

---

## 4. Token Handling <a id='token-handling'></a>

### Token Types Overview

| Token | Purpose | Lifetime | Storage |
|-------|---------|----------|--------|
| **Access Token** | API authorization | Short (minutes to hours) | Memory/Secure storage |
| **Refresh Token** | Obtain new access tokens | Long (days to months) | Secure server-side storage |
| **ID Token** | User authentication | Short (minutes) | Memory only |

### JWT (JSON Web Token) Structure

```
┌─────────────────────────────────────────────────────────────┐
│                        JWT Structure                         │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  Header.Payload.Signature                                    │
│                                                              │
│  ┌─────────────┐  ┌─────────────────┐  ┌────────────────┐   │
│  │   HEADER    │  │     PAYLOAD     │  │   SIGNATURE    │   │
│  │  (Base64)   │  │    (Base64)     │  │   (Base64)     │   │
│  ├─────────────┤  ├─────────────────┤  ├────────────────┤   │
│  │ {           │  │ {               │  │                │   │
│  │  "alg":"RS256"│ │  "sub":"123",   │  │ HMAC-SHA256(   │   │
│  │  "typ":"JWT" │  │  "exp":1234567, │  │   base64(hdr)  │   │
│  │  "kid":"key1"│  │  "iat":1234560  │  │   + "." +      │   │
│  │ }           │  │ }               │  │   base64(pld), │   │
│  │             │  │                 │  │   secret       │   │
│  │             │  │                 │  │ )              │   │
│  └─────────────┘  └─────────────────┘  └────────────────┘   │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# JWT Creation and Verification with PyJWT
# pip install PyJWT cryptography

import jwt
from datetime import datetime, timedelta, timezone
from typing import Dict, Any, Optional

class JWTHandler:
    """
    JWT token creation and verification handler.
    """
    
    def __init__(self, secret_key: str, algorithm: str = 'HS256'):
        """
        Initialize JWT handler.
        
        Args:
            secret_key: Secret key for signing/verification
            algorithm: JWT algorithm (HS256, RS256, etc.)
        """
        self.secret_key = secret_key
        self.algorithm = algorithm
    
    def create_access_token(
        self,
        subject: str,
        expires_delta: timedelta = timedelta(minutes=30),
        additional_claims: Dict[str, Any] = None
    ) -> str:
        """
        Create a new access token.
        
        Args:
            subject: Token subject (usually user ID)
            expires_delta: Token expiration time
            additional_claims: Extra claims to include
        
        Returns:
            Encoded JWT string
        """
        now = datetime.now(timezone.utc)
        
        payload = {
            'sub': subject,
            'iat': now,
            'exp': now + expires_delta,
            'type': 'access'
        }
        
        if additional_claims:
            payload.update(additional_claims)
        
        return jwt.encode(payload, self.secret_key, algorithm=self.algorithm)
    
    def create_refresh_token(
        self,
        subject: str,
        expires_delta: timedelta = timedelta(days=30)
    ) -> str:
        """
        Create a new refresh token.
        
        Args:
            subject: Token subject (usually user ID)
            expires_delta: Token expiration time
        
        Returns:
            Encoded JWT string
        """
        now = datetime.now(timezone.utc)
        
        payload = {
            'sub': subject,
            'iat': now,
            'exp': now + expires_delta,
            'type': 'refresh'
        }
        
        return jwt.encode(payload, self.secret_key, algorithm=self.algorithm)
    
    def verify_token(self, token: str) -> Optional[Dict[str, Any]]:
        """
        Verify and decode a JWT token.
        
        Args:
            token: JWT token string
        
        Returns:
            Decoded payload if valid, None if invalid
        """
        try:
            payload = jwt.decode(
                token,
                self.secret_key,
                algorithms=[self.algorithm]
            )
            return payload
        except jwt.ExpiredSignatureError:
            print("Token has expired")
            return None
        except jwt.InvalidTokenError as e:
            print(f"Invalid token: {e}")
            return None


# Example usage
print("JWT Token Handler Example")
print("="*50)

jwt_handler = JWTHandler(secret_key='your-256-bit-secret-key-here')

# Create tokens
access_token = jwt_handler.create_access_token(
    subject='user_123',
    additional_claims={'roles': ['user', 'admin'], 'email': 'user@example.com'}
)

refresh_token = jwt_handler.create_refresh_token(subject='user_123')

print(f"\nAccess Token (first 80 chars):\n{access_token[:80]}...")
print(f"\nRefresh Token (first 80 chars):\n{refresh_token[:80]}...")

# Verify token
payload = jwt_handler.verify_token(access_token)
print(f"\nDecoded Payload: {payload}")

In [ ]:
# Token Refresh Flow Implementation
import requests
from typing import Tuple, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class TokenSet:
    """Container for OAuth tokens."""
    access_token: str
    refresh_token: str
    expires_at: datetime
    token_type: str = 'Bearer'
    scope: str = ''


class TokenManager:
    """
    Manages OAuth tokens with automatic refresh.
    """
    
    def __init__(
        self,
        token_endpoint: str,
        client_id: str,
        client_secret: str = None,
        refresh_threshold_seconds: int = 300
    ):
        """
        Initialize token manager.
        
        Args:
            token_endpoint: OAuth token endpoint
            client_id: OAuth client ID
            client_secret: OAuth client secret (optional for public clients)
            refresh_threshold_seconds: Refresh token this many seconds before expiry
        """
        self.token_endpoint = token_endpoint
        self.client_id = client_id
        self.client_secret = client_secret
        self.refresh_threshold = timedelta(seconds=refresh_threshold_seconds)
        self._tokens: Optional[TokenSet] = None
    
    def set_tokens(self, token_response: dict) -> TokenSet:
        """
        Store tokens from an OAuth token response.
        
        Args:
            token_response: Response from token endpoint
        
        Returns:
            TokenSet object
        """
        expires_in = token_response.get('expires_in', 3600)
        
        self._tokens = TokenSet(
            access_token=token_response['access_token'],
            refresh_token=token_response.get('refresh_token', ''),
            expires_at=datetime.now() + timedelta(seconds=expires_in),
            token_type=token_response.get('token_type', 'Bearer'),
            scope=token_response.get('scope', '')
        )
        
        return self._tokens
    
    def needs_refresh(self) -> bool:
        """
        Check if access token needs to be refreshed.
        """
        if self._tokens is None:
            return True
        return datetime.now() + self.refresh_threshold >= self._tokens.expires_at
    
    def refresh_access_token(self) -> TokenSet:
        """
        Refresh the access token using the refresh token.
        
        Returns:
            New TokenSet with refreshed tokens
        
        Raises:
            ValueError: If no refresh token is available
            requests.HTTPError: If refresh fails
        """
        if not self._tokens or not self._tokens.refresh_token:
            raise ValueError("No refresh token available")
        
        payload = {
            'grant_type': 'refresh_token',
            'refresh_token': self._tokens.refresh_token,
            'client_id': self.client_id
        }
        
        if self.client_secret:
            payload['client_secret'] = self.client_secret
        
        response = requests.post(
            self.token_endpoint,
            data=payload,
            headers={'Content-Type': 'application/x-www-form-urlencoded'}
        )
        
        response.raise_for_status()
        return self.set_tokens(response.json())
    
    def get_valid_access_token(self) -> str:
        """
        Get a valid access token, refreshing if necessary.
        
        Returns:
            Valid access token string
        """
        if self.needs_refresh() and self._tokens and self._tokens.refresh_token:
            self.refresh_access_token()
        
        if self._tokens is None:
            raise ValueError("No tokens available")
        
        return self._tokens.access_token


print("Token Manager - Automatic Refresh")
print("="*50)
print("""
Usage:
    manager = TokenManager(
        token_endpoint='https://auth.example.com/oauth/token',
        client_id='my-client-id',
        client_secret='my-secret'
    )
    
    # After initial authentication
    manager.set_tokens(initial_token_response)
    
    # Get valid token (auto-refreshes if needed)
    token = manager.get_valid_access_token()
""")

In [ ]:
# Token Introspection and Revocation
import requests
from typing import Dict, Any, Optional

class TokenIntrospection:
    """
    OAuth 2.0 Token Introspection (RFC 7662) implementation.
    """
    
    def __init__(
        self,
        introspection_endpoint: str,
        client_id: str,
        client_secret: str
    ):
        self.introspection_endpoint = introspection_endpoint
        self.client_id = client_id
        self.client_secret = client_secret
    
    def introspect(self, token: str, token_type_hint: str = 'access_token') -> Dict[str, Any]:
        """
        Introspect a token to check its validity and get metadata.
        
        Args:
            token: The token to introspect
            token_type_hint: 'access_token' or 'refresh_token'
        
        Returns:
            Introspection response with 'active' boolean and claims
        """
        response = requests.post(
            self.introspection_endpoint,
            data={
                'token': token,
                'token_type_hint': token_type_hint
            },
            auth=(self.client_id, self.client_secret)
        )
        
        response.raise_for_status()
        return response.json()


class TokenRevocation:
    """
    OAuth 2.0 Token Revocation (RFC 7009) implementation.
    """
    
    def __init__(
        self,
        revocation_endpoint: str,
        client_id: str,
        client_secret: str = None
    ):
        self.revocation_endpoint = revocation_endpoint
        self.client_id = client_id
        self.client_secret = client_secret
    
    def revoke(self, token: str, token_type_hint: str = 'refresh_token') -> bool:
        """
        Revoke a token (logout functionality).
        
        Args:
            token: The token to revoke
            token_type_hint: 'access_token' or 'refresh_token'
        
        Returns:
            True if revocation was successful
        """
        data = {
            'token': token,
            'token_type_hint': token_type_hint,
            'client_id': self.client_id
        }
        
        if self.client_secret:
            data['client_secret'] = self.client_secret
        
        response = requests.post(self.revocation_endpoint, data=data)
        
        # RFC 7009: 200 OK indicates successful revocation
        return response.status_code == 200


# Example introspection response
print("Token Introspection Response Example")
print("="*50)

introspection_response = {
    'active': True,
    'client_id': 'my-client-id',
    'username': 'johndoe',
    'scope': 'openid profile email',
    'sub': 'user_123456',
    'aud': 'my-client-id',
    'iss': 'https://auth.example.com',
    'exp': 1704100000,
    'iat': 1704096400,
    'token_type': 'Bearer'
}

print("\nActive Token Response:")
for key, value in introspection_response.items():
    print(f"  {key}: {value}")

print("\nInactive/Revoked Token Response:")
print("  { 'active': False }")

---

## 5. Security Best Practices <a id='security-best-practices'></a>

### Token Security

| Practice | Description |
|----------|-------------|
| **Use HTTPS** | Always use TLS for all OAuth endpoints |
| **Short-lived access tokens** | 5-60 minutes recommended |
| **Rotate refresh tokens** | Issue new refresh token on each use |
| **Secure storage** | Never store tokens in localStorage for sensitive apps |
| **Token binding** | Bind tokens to specific clients/sessions |

### PKCE Everywhere

```
✅ Always use PKCE for:
   • Single Page Applications (SPAs)
   • Mobile Applications
   • Desktop Applications
   • Server-side Applications (optional but recommended)

❌ Never use (deprecated):
   • Implicit Grant
   • Resource Owner Password Credentials
```

### State Parameter

Always use and validate the `state` parameter to prevent CSRF attacks:

1. Generate cryptographically random state
2. Store in session before redirect
3. Validate on callback matches stored value
4. Clear after validation

In [ ]:
# Security Best Practices Implementation
import secrets
import hashlib
from typing import Optional
from datetime import datetime, timedelta

class SecureTokenStorage:
    """
    Secure token storage with encryption and expiry.
    In production, use proper secrets management (Vault, AWS Secrets Manager, etc.)
    """
    
    def __init__(self):
        self._tokens = {}  # In-memory storage (use Redis/database in production)
    
    def store_token(
        self,
        user_id: str,
        token: str,
        token_type: str,
        expires_in: int
    ) -> str:
        """
        Securely store a token.
        
        Args:
            user_id: User identifier
            token: The token to store
            token_type: 'access' or 'refresh'
            expires_in: Seconds until expiry
        
        Returns:
            Token reference ID (use this instead of the actual token in cookies)
        """
        # Create a reference ID (store this in cookie instead of actual token)
        reference_id = secrets.token_urlsafe(32)
        
        # Hash the token for storage (optional additional security layer)
        token_hash = hashlib.sha256(token.encode()).hexdigest()
        
        self._tokens[reference_id] = {
            'user_id': user_id,
            'token': token,  # In production, encrypt this
            'token_hash': token_hash,
            'token_type': token_type,
            'expires_at': datetime.now() + timedelta(seconds=expires_in),
            'created_at': datetime.now()
        }
        
        return reference_id
    
    def get_token(self, reference_id: str) -> Optional[str]:
        """
        Retrieve a token by reference ID.
        
        Args:
            reference_id: The token reference ID
        
        Returns:
            The token if valid and not expired, None otherwise
        """
        token_data = self._tokens.get(reference_id)
        
        if not token_data:
            return None
        
        if datetime.now() >= token_data['expires_at']:
            # Token expired, remove it
            del self._tokens[reference_id]
            return None
        
        return token_data['token']
    
    def revoke_token(self, reference_id: str) -> bool:
        """
        Revoke a token.
        """
        if reference_id in self._tokens:
            del self._tokens[reference_id]
            return True
        return False
    
    def revoke_all_user_tokens(self, user_id: str) -> int:
        """
        Revoke all tokens for a user (logout from all devices).
        
        Returns:
            Number of tokens revoked
        """
        to_revoke = [
            ref_id for ref_id, data in self._tokens.items()
            if data['user_id'] == user_id
        ]
        
        for ref_id in to_revoke:
            del self._tokens[ref_id]
        
        return len(to_revoke)


# State Management for CSRF Protection
class StateManager:
    """
    Manages OAuth state parameter for CSRF protection.
    """
    
    def __init__(self, max_age_seconds: int = 600):
        self._states = {}  # Use Redis in production
        self.max_age = timedelta(seconds=max_age_seconds)
    
    def generate_state(self, session_id: str, metadata: dict = None) -> str:
        """
        Generate a new state parameter.
        
        Args:
            session_id: The user's session ID
            metadata: Optional metadata to store with state
        
        Returns:
            Cryptographically secure state string
        """
        state = secrets.token_urlsafe(32)
        
        self._states[state] = {
            'session_id': session_id,
            'created_at': datetime.now(),
            'metadata': metadata or {}
        }
        
        return state
    
    def validate_state(self, state: str, session_id: str) -> tuple:
        """
        Validate a state parameter.
        
        Args:
            state: The state parameter from callback
            session_id: The current session ID
        
        Returns:
            Tuple of (is_valid, metadata)
        """
        state_data = self._states.pop(state, None)  # Remove after use
        
        if not state_data:
            return False, {'error': 'State not found'}
        
        # Check expiry
        if datetime.now() - state_data['created_at'] > self.max_age:
            return False, {'error': 'State expired'}
        
        # Check session binding
        if state_data['session_id'] != session_id:
            return False, {'error': 'Session mismatch'}
        
        return True, state_data['metadata']


print("Security Best Practices")
print("="*50)
print("""
1. Token Storage:
   - Store reference IDs in cookies, not actual tokens
   - Use HttpOnly, Secure, SameSite=Strict cookies
   - Server-side token storage with encryption

2. State Parameter:
   - Always generate cryptographically random state
   - Bind to user session
   - Validate and consume on callback
   - Set reasonable expiration (5-10 minutes)

3. Token Lifecycle:
   - Short-lived access tokens (5-30 minutes)
   - Longer refresh tokens (hours to days)
   - Implement token rotation
   - Support revocation for logout
""")

---

## 6. Python Implementation Examples <a id='python-examples'></a>

### Complete OAuth Client Implementation

In [ ]:
# Complete OAuth 2.0 / OIDC Client Implementation
import requests
import secrets
import hashlib
import base64
import urllib.parse
from dataclasses import dataclass, field
from typing import Dict, Any, Optional, List
from datetime import datetime, timedelta
import json

@dataclass
class OAuthConfig:
    """OAuth/OIDC provider configuration."""
    client_id: str
    client_secret: Optional[str]
    authorization_endpoint: str
    token_endpoint: str
    redirect_uri: str
    scopes: List[str] = field(default_factory=lambda: ['openid'])
    userinfo_endpoint: Optional[str] = None
    revocation_endpoint: Optional[str] = None
    use_pkce: bool = True


@dataclass
class AuthSession:
    """Stores authentication session data."""
    state: str
    code_verifier: Optional[str] = None
    nonce: Optional[str] = None
    created_at: datetime = field(default_factory=datetime.now)


class OAuthClient:
    """
    Complete OAuth 2.0 / OpenID Connect client implementation.
    Supports Authorization Code flow with PKCE.
    """
    
    def __init__(self, config: OAuthConfig):
        """
        Initialize the OAuth client.
        
        Args:
            config: OAuth configuration object
        """
        self.config = config
        self._sessions: Dict[str, AuthSession] = {}
    
    def start_authorization(self, additional_params: Dict[str, str] = None) -> tuple:
        """
        Start the authorization flow.
        
        Args:
            additional_params: Extra parameters to include in authorization URL
        
        Returns:
            Tuple of (authorization_url, state)
        """
        # Generate state for CSRF protection
        state = secrets.token_urlsafe(32)
        
        # Create session
        session = AuthSession(state=state)
        
        # Build authorization parameters
        params = {
            'response_type': 'code',
            'client_id': self.config.client_id,
            'redirect_uri': self.config.redirect_uri,
            'scope': ' '.join(self.config.scopes),
            'state': state
        }
        
        # Add PKCE if enabled
        if self.config.use_pkce:
            code_verifier = secrets.token_urlsafe(64)[:64]
            code_challenge = base64.urlsafe_b64encode(
                hashlib.sha256(code_verifier.encode()).digest()
            ).decode().rstrip('=')
            
            params['code_challenge'] = code_challenge
            params['code_challenge_method'] = 'S256'
            session.code_verifier = code_verifier
        
        # Add nonce for OIDC
        if 'openid' in self.config.scopes:
            nonce = secrets.token_urlsafe(32)
            params['nonce'] = nonce
            session.nonce = nonce
        
        # Add any additional parameters
        if additional_params:
            params.update(additional_params)
        
        # Store session
        self._sessions[state] = session
        
        # Build URL
        auth_url = f"{self.config.authorization_endpoint}?{urllib.parse.urlencode(params)}"
        
        return auth_url, state
    
    def handle_callback(
        self,
        code: str,
        state: str,
        error: str = None,
        error_description: str = None
    ) -> Dict[str, Any]:
        """
        Handle the OAuth callback.
        
        Args:
            code: Authorization code from callback
            state: State parameter from callback
            error: Error code if authorization failed
            error_description: Error description
        
        Returns:
            Token response dictionary
        
        Raises:
            ValueError: If state is invalid or authorization failed
        """
        # Check for errors
        if error:
            raise ValueError(f"Authorization failed: {error} - {error_description}")
        
        # Validate state
        session = self._sessions.pop(state, None)
        if not session:
            raise ValueError("Invalid state parameter - possible CSRF attack")
        
        # Check session age (10 minute timeout)
        if datetime.now() - session.created_at > timedelta(minutes=10):
            raise ValueError("Authorization session expired")
        
        # Exchange code for tokens
        return self._exchange_code(code, session)
    
    def _exchange_code(self, code: str, session: AuthSession) -> Dict[str, Any]:
        """
        Exchange authorization code for tokens.
        """
        payload = {
            'grant_type': 'authorization_code',
            'client_id': self.config.client_id,
            'code': code,
            'redirect_uri': self.config.redirect_uri
        }
        
        # Add client secret if confidential client
        if self.config.client_secret:
            payload['client_secret'] = self.config.client_secret
        
        # Add code verifier if PKCE was used
        if session.code_verifier:
            payload['code_verifier'] = session.code_verifier
        
        response = requests.post(
            self.config.token_endpoint,
            data=payload,
            headers={'Content-Type': 'application/x-www-form-urlencoded'}
        )
        
        response.raise_for_status()
        return response.json()
    
    def refresh_token(self, refresh_token: str) -> Dict[str, Any]:
        """
        Refresh an access token.
        
        Args:
            refresh_token: The refresh token
        
        Returns:
            New token response
        """
        payload = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'client_id': self.config.client_id
        }
        
        if self.config.client_secret:
            payload['client_secret'] = self.config.client_secret
        
        response = requests.post(
            self.config.token_endpoint,
            data=payload,
            headers={'Content-Type': 'application/x-www-form-urlencoded'}
        )
        
        response.raise_for_status()
        return response.json()
    
    def get_userinfo(self, access_token: str) -> Dict[str, Any]:
        """
        Get user information from the UserInfo endpoint.
        
        Args:
            access_token: Valid access token
        
        Returns:
            User information claims
        """
        if not self.config.userinfo_endpoint:
            raise ValueError("UserInfo endpoint not configured")
        
        response = requests.get(
            self.config.userinfo_endpoint,
            headers={'Authorization': f'Bearer {access_token}'}
        )
        
        response.raise_for_status()
        return response.json()
    
    def revoke_token(self, token: str, token_type_hint: str = 'refresh_token') -> bool:
        """
        Revoke a token.
        
        Args:
            token: Token to revoke
            token_type_hint: 'access_token' or 'refresh_token'
        
        Returns:
            True if revocation was successful
        """
        if not self.config.revocation_endpoint:
            raise ValueError("Revocation endpoint not configured")
        
        payload = {
            'token': token,
            'token_type_hint': token_type_hint,
            'client_id': self.config.client_id
        }
        
        if self.config.client_secret:
            payload['client_secret'] = self.config.client_secret
        
        response = requests.post(self.config.revocation_endpoint, data=payload)
        return response.status_code == 200


# Example usage
print("Complete OAuth 2.0 / OIDC Client")
print("="*50)

# Configure the client
config = OAuthConfig(
    client_id='my-app-client-id',
    client_secret='my-app-client-secret',  # None for public clients
    authorization_endpoint='https://auth.example.com/authorize',
    token_endpoint='https://auth.example.com/oauth/token',
    redirect_uri='https://myapp.com/callback',
    scopes=['openid', 'profile', 'email'],
    userinfo_endpoint='https://auth.example.com/userinfo',
    revocation_endpoint='https://auth.example.com/oauth/revoke',
    use_pkce=True
)

client = OAuthClient(config)

# Start authorization
auth_url, state = client.start_authorization()
print(f"\n1. Redirect user to:\n{auth_url[:100]}...")
print(f"\n2. Store state in session: {state[:20]}...")
print("""
3. After callback:
   tokens = client.handle_callback(
       code=request.args['code'],
       state=request.args['state']
   )

4. Get user info:
   userinfo = client.get_userinfo(tokens['access_token'])
""")

In [ ]:
# FastAPI Integration Example
from typing import Optional

fastapi_example = '''
from fastapi import FastAPI, Request, HTTPException, Depends
from fastapi.responses import RedirectResponse
from starlette.middleware.sessions import SessionMiddleware
import httpx

app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key="your-secret-key")

# OAuth Configuration
OAUTH_CONFIG = {
    "client_id": "your-client-id",
    "client_secret": "your-client-secret",
    "authorization_endpoint": "https://auth.example.com/authorize",
    "token_endpoint": "https://auth.example.com/oauth/token",
    "userinfo_endpoint": "https://auth.example.com/userinfo",
    "redirect_uri": "http://localhost:8000/callback",
    "scopes": ["openid", "profile", "email"]
}


@app.get("/login")
async def login(request: Request):
    """Initiate OAuth login flow."""
    import secrets
    import hashlib
    import base64
    
    # Generate PKCE verifier and challenge
    code_verifier = secrets.token_urlsafe(64)[:64]
    code_challenge = base64.urlsafe_b64encode(
        hashlib.sha256(code_verifier.encode()).digest()
    ).decode().rstrip("=")
    
    # Generate state
    state = secrets.token_urlsafe(32)
    
    # Store in session
    request.session["oauth_state"] = state
    request.session["code_verifier"] = code_verifier
    
    # Build authorization URL
    params = {
        "response_type": "code",
        "client_id": OAUTH_CONFIG["client_id"],
        "redirect_uri": OAUTH_CONFIG["redirect_uri"],
        "scope": " ".join(OAUTH_CONFIG["scopes"]),
        "state": state,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256"
    }
    
    auth_url = f"{OAUTH_CONFIG['authorization_endpoint']}?{urlencode(params)}"
    return RedirectResponse(auth_url)


@app.get("/callback")
async def callback(request: Request, code: str, state: str):
    """Handle OAuth callback."""
    # Validate state
    if state != request.session.get("oauth_state"):
        raise HTTPException(status_code=400, detail="Invalid state")
    
    code_verifier = request.session.pop("code_verifier", None)
    request.session.pop("oauth_state", None)
    
    # Exchange code for tokens
    async with httpx.AsyncClient() as client:
        token_response = await client.post(
            OAUTH_CONFIG["token_endpoint"],
            data={
                "grant_type": "authorization_code",
                "client_id": OAUTH_CONFIG["client_id"],
                "client_secret": OAUTH_CONFIG["client_secret"],
                "code": code,
                "redirect_uri": OAUTH_CONFIG["redirect_uri"],
                "code_verifier": code_verifier
            }
        )
        tokens = token_response.json()
    
    # Store tokens in session (or database)
    request.session["access_token"] = tokens["access_token"]
    request.session["refresh_token"] = tokens.get("refresh_token")
    
    return RedirectResponse("/protected")


async def get_current_user(request: Request):
    """Dependency to get current authenticated user."""
    access_token = request.session.get("access_token")
    if not access_token:
        raise HTTPException(status_code=401, detail="Not authenticated")
    
    async with httpx.AsyncClient() as client:
        userinfo_response = await client.get(
            OAUTH_CONFIG["userinfo_endpoint"],
            headers={"Authorization": f"Bearer {access_token}"}
        )
        if userinfo_response.status_code != 200:
            raise HTTPException(status_code=401, detail="Invalid token")
        return userinfo_response.json()


@app.get("/protected")
async def protected_route(user: dict = Depends(get_current_user)):
    """Protected route requiring authentication."""
    return {"message": f"Hello, {user.get('name', 'User')}!", "user": user}


@app.get("/logout")
async def logout(request: Request):
    """Logout and clear session."""
    request.session.clear()
    return RedirectResponse("/")
'''

print("FastAPI OAuth Integration Example")
print("="*50)
print(fastapi_example)

---

## Summary

### Key Takeaways

1. **OAuth 2.0** is for **authorization** (what can you access)
2. **OpenID Connect** adds **authentication** (who are you)
3. **Always use PKCE** for public clients (SPAs, mobile apps)
4. **Keep access tokens short-lived**, use refresh tokens for renewal
5. **Validate the state parameter** to prevent CSRF attacks
6. **Verify ID tokens** before trusting claims
7. **Store tokens securely** - never in localStorage for sensitive apps

### Quick Reference

| Scenario | Grant Type | PKCE |
|----------|------------|------|
| Web app with backend | Authorization Code | Optional |
| SPA (React, Vue, Angular) | Authorization Code + PKCE | Required |
| Mobile app | Authorization Code + PKCE | Required |
| Server-to-server | Client Credentials | N/A |
| CLI tool | Device Code or Client Credentials | Depends |

### Useful Resources

- [OAuth 2.0 RFC 6749](https://tools.ietf.org/html/rfc6749)
- [PKCE RFC 7636](https://tools.ietf.org/html/rfc7636)
- [OpenID Connect Core 1.0](https://openid.net/specs/openid-connect-core-1_0.html)
- [OAuth 2.0 Security Best Practices](https://tools.ietf.org/html/draft-ietf-oauth-security-topics)
- [JWT RFC 7519](https://tools.ietf.org/html/rfc7519)